# Washington DC Biking data | Hourly Bike Count Prediction

## 2. Data Preparation & Feature Engineering
MBD O-1-5

### Notebook preperation

In [84]:
%matplotlib inline
%load_ext nb_black
# To automatically reload the function file
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [85]:
# Data Import
# hourly_raw_data = dd.read_csv('gs://bikeshareproject/hour.csv') #File Stored on Google Cloud Storage bucket "bikeshareproject"
hourly_raw_data = dd.read_csv("hour.csv")

hourly_raw_data_pd = pd.read_csv("hour.csv")

<IPython.core.display.Javascript object>

In [86]:
# hourly_raw_data = dd.read_csv('hour.csv')

<IPython.core.display.Javascript object>

In [87]:
hourly_raw_data["dteday"].map_partitions(
    pd.to_datetime, format="%Y-%m-%d", meta=("datetime64[ns]")
)

Dask Series Structure:
npartitions=1
    datetime64[ns]
               ...
dtype: datetime64[ns]
Dask Name: to_datetime, 5 tasks

<IPython.core.display.Javascript object>

In [88]:
hourly_raw_data.compute().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


<IPython.core.display.Javascript object>

In [89]:
hourly_raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


<IPython.core.display.Javascript object>

In [90]:
#hourly_raw_data.visualize()

<IPython.core.display.Javascript object>

# Feature Engineering

### Converting `dteday` to date

## Add `isDaylight` and  `isNoon` for hourly data

Astral module is used to calculate flags for daylight and noon time. 

A customized function is defined to classify a row as daylight. If the hour of a record is less than the hour of sunset in Washington DC and more than the time of sunrise, it is flagged as daylight, otherwise it is flagged as not daylight.
Noon time flag is also created using a customized function. If the hour of a record is equal to the hour of noon in Washington DC, it is flagged as noon, otherwise it is flagged as not noon. 

In [91]:
hourly_raw_data["isDaylight"] = (hourly_raw_data["hr"] > 6) & (
    hourly_raw_data["hr"] < 20
)

hourly_raw_data["isNoon"] = (hourly_raw_data["hr"] > 10) & (hourly_raw_data["hr"] < 14)



<IPython.core.display.Javascript object>

### Adding the temp atemp windspeed hum relative to the last 7 days value

In exploratory data analysis, it was found that there are outliers in seasonal variables. In order to make a robust model that is able to predict outliers, new variables are created for temp, atemp, hum, and wind speed. In case of temp, mean of temp for the last seven days is deducted from current temp value and the resulting value is divided by standard deviation of temp for the last seven days.

In [92]:
to_relative = ["temp", "atemp", "hum", "windspeed"]
hourly_raw_data = relative_values(hourly_raw_data, to_relative)


<IPython.core.display.Javascript object>

In [93]:
hourly_raw_data.compute()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,...,windspeed,casual,registered,cnt,isDaylight,isNoon,relative_temp,relative_atemp,relative_hum,relative_windspeed
6,7,2011-01-01,1,0,1,6,0,6,0,1,...,0.0000,2,0,2,False,False,-1.290994,-0.410671,0.792594,-0.408248
7,8,2011-01-01,1,0,1,7,0,6,0,1,...,0.0000,1,2,3,True,False,-2.939874,-1.697564,2.828427,-0.377964
8,9,2011-01-01,1,0,1,8,0,6,0,1,...,0.0000,1,7,8,True,False,0.840027,1.049718,-1.037749,-0.353553
9,10,2011-01-01,1,0,1,9,0,6,0,1,...,0.0000,8,6,14,True,False,6.270697,5.732372,-0.664784,-0.333333
10,11,2011-01-01,1,0,1,10,0,6,0,1,...,0.2537,12,24,36,True,False,4.451044,4.284212,-0.619389,8.637677
11,12,2011-01-01,1,0,1,11,0,6,0,1,...,0.2836,26,30,56,True,True,2.080626,0.964934,0.810243,3.213730
12,13,2011-01-01,1,0,1,12,0,6,0,1,...,0.2836,29,55,84,True,True,2.708013,3.103062,-0.378272,2.214437
13,14,2011-01-01,1,0,1,13,0,6,0,2,...,0.2985,47,47,94,True,True,2.610559,2.797839,-1.835309,1.922405
14,15,2011-01-01,1,0,1,14,0,6,0,2,...,0.2836,35,71,106,True,False,2.041551,2.133943,-1.579824,1.523345
15,16,2011-01-01,1,0,1,15,0,6,0,2,...,0.2985,40,70,110,True,False,1.521491,1.583008,-0.104379,1.476610


<IPython.core.display.Javascript object>

### Adding `RushHour-High` & 	`RushHour-Med`  &	`RushHour-Low`

The interactive time series shows that there are variations in casual, registered, and total bikers during the span of a day. This realization led to creation of a rush hour flag. The logic for this flag is as follows:

Working Day:  
10:00 AM to 6:00 PM is flagged as high rush hour. 7:00 PM to Midnight and 08:00 AM to 09:00 AM are flagged as medium rush hour. Whereas, all other hours are flagged as low rush hour.  
Holiday:  
7:00 AM to 9:00 AM and 4:00 to 8:00 PM is flagged as high rush hour. 6:00 AM, 10:00 AM till 1:00 PM,  3:00 PM, and 9:00 PM till 11:00 PM are flagged as medium rush hour. Whereas, all other hours are flagged as low rush hour.

In [94]:
hourly_raw_data["Wday_RushHour_High"] = (
    (hourly_raw_data["workingday"] == 0)
    & (hourly_raw_data["hr"] > 10)
    & (hourly_raw_data["hr"] < 18)
)
hourly_raw_data["Wday_RushHour_Med"] = (hourly_raw_data["workingday"] == 0) & (
    ((hourly_raw_data["hr"] > 19) & (hourly_raw_data["hr"] < 24))
    | ((hourly_raw_data["hr"] > 8) & (hourly_raw_data["hr"] < 9))
)
hourly_raw_data["Wday_RushHour_Low"] = (hourly_raw_data["workingday"] == 1) & (
    (hourly_raw_data["hr"] > 0) & (hourly_raw_data["hr"] < 7)
)

<IPython.core.display.Javascript object>

In [95]:
hourly_raw_data["Wend_RushHour_High"] = (hourly_raw_data["workingday"] == 1) & (
    ((hourly_raw_data["hr"] > 6) & (hourly_raw_data["hr"] < 10))
    | ((hourly_raw_data["hr"] > 16) & (hourly_raw_data["hr"] < 20))
)
hourly_raw_data["Wend_RushHour_Med"] = (hourly_raw_data["workingday"] == 1) & (
    ((hourly_raw_data["hr"] > 13) & (hourly_raw_data["hr"] < 15))
    | ((hourly_raw_data["hr"] > 21) & (hourly_raw_data["hr"] < 23))
)
hourly_raw_data["Wend_RushHour_Low"] = (hourly_raw_data["workingday"] == 1) & (
    (hourly_raw_data["hr"] > 0) & (hourly_raw_data["hr"] < 5)
)

<IPython.core.display.Javascript object>

In [96]:
hourly_raw_data.compute().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17373 entries, 6 to 17378
Data columns (total 29 columns):
instant               17373 non-null int64
dteday                17373 non-null object
season                17373 non-null int64
yr                    17373 non-null int64
mnth                  17373 non-null int64
hr                    17373 non-null int64
holiday               17373 non-null int64
weekday               17373 non-null int64
workingday            17373 non-null int64
weathersit            17373 non-null int64
temp                  17373 non-null float64
atemp                 17373 non-null float64
hum                   17373 non-null float64
windspeed             17373 non-null float64
casual                17373 non-null int64
registered            17373 non-null int64
cnt                   17373 non-null int64
isDaylight            17373 non-null bool
isNoon                17373 non-null bool
relative_temp         17373 non-null float64
relative_atemp        

<IPython.core.display.Javascript object>

## Splitting Data

In [97]:
workingdays = num_name(hourly_raw_data.loc[(hourly_raw_data['workingday'].isin([1]) )])
holidays = num_name(hourly_raw_data.loc[(~hourly_raw_data['workingday'].isin([1]) )])

/Users/rafaelhernandez/miniconda3/envs/intro_python/lib/python3.7/site-packages/dask/dataframe/core.py:2345: UserWarning:


You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('season', 'object'))


/Users/rafaelhernandez/miniconda3/envs/intro_python/lib/python3.7/site-packages/dask/dataframe/core.py:2345: UserWarning:


You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func,

<IPython.core.display.Javascript object>

### One hot Encoding | 2x for splitted datasets
For `season`, `weathersit`, `mnth`,`weekday`,`hr`

In [98]:
category  = ['season', 'weathersit', 'mnth','weekday','hr']

pipeline = make_pipeline(Categorizer(columns=category), DummyEncoder(columns=category))

<IPython.core.display.Javascript object>

In [99]:
workingdays = pipeline.fit_transform(workingdays)


<IPython.core.display.Javascript object>

In [100]:
holidays = pipeline.fit_transform(holidays)


<IPython.core.display.Javascript object>

# Final Datasets

In [101]:
holidays.compute().info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5508 entries, 6 to 17354
Data columns (total 75 columns):
instant                  5508 non-null int64
dteday                   5508 non-null object
yr                       5508 non-null int64
holiday                  5508 non-null int64
workingday               5508 non-null int64
temp                     5508 non-null float64
atemp                    5508 non-null float64
hum                      5508 non-null float64
windspeed                5508 non-null float64
casual                   5508 non-null int64
registered               5508 non-null int64
cnt                      5508 non-null int64
isDaylight               5508 non-null bool
isNoon                   5508 non-null bool
relative_temp            5508 non-null float64
relative_atemp           5508 non-null float64
relative_hum             5508 non-null float64
relative_windspeed       5508 non-null float64
Wday_RushHour_High       5508 non-null bool
Wday_RushHour_Med      

<IPython.core.display.Javascript object>

## Save Both datasets

In [102]:
workingdays.compute().to_csv("workingdays_data_prepared.csv", index=False)
holidays.compute().to_csv("weekends_holi_data_prepared.csv", index=False)

<IPython.core.display.Javascript object>